In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_15.csv",r"B:\project\ai\ai_chunk_10.csv","B:\project\human\human_chunk_16.csv",r"B:\project\ai\ai_chunk_9.csv","B:\project\human\human_chunk_17.csv",r"B:\project\ai\ai_chunk_8.csv","B:\project\human\human_chunk_18.csv",r"B:\project\ai\ai_chunk_7.csv","B:\project\human\human_chunk_19.csv",r"B:\project\ai\ai_chunk_6.csv","B:\project\human\human_chunk_20.csv",r"B:\project\ai\ai_chunk_5.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)

(183477, 2)


In [4]:
from transformers import LongformerTokenizer
data=df['text'].tolist()
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokens=tokenizer(data,return_tensors='pt',padding=True,truncation=True)

In [5]:
from sklearn.model_selection import train_test_split
x=tokens['input_ids']
y=df['generated']
xtrain,xtemp,ytrain,ytemp=train_test_split(x,y,test_size=0.3,random_state=42)
xtest,xval,ytest,yval=train_test_split(xtemp,ytemp,test_size=0.3,random_state=42)
xtest,xtrain,xval=xtest.numpy(),xtrain.numpy(),xval.numpy()

In [7]:
from keras.models import load_model
model=load_model('finetuned4.keras')
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=5,batch_size=256,validation_data=(xval,yval))

Epoch 1/5
502/502 [==============================] - 375s 746ms/step - loss: 0.0023 - accuracy: 0.9995 - val_loss: 0.0015 - val_accuracy: 0.9995
Epoch 2/5
502/502 [==============================] - 334s 664ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 0.0027 - val_accuracy: 0.9992
Epoch 3/5
502/502 [==============================] - 321s 640ms/step - loss: 6.9958e-04 - accuracy: 0.9999 - val_loss: 0.0026 - val_accuracy: 0.9995
Epoch 4/5
502/502 [==============================] - 320s 638ms/step - loss: 4.7553e-04 - accuracy: 0.9999 - val_loss: 0.0023 - val_accuracy: 0.9995
Epoch 5/5
502/502 [==============================] - 320s 638ms/step - loss: 5.7435e-04 - accuracy: 0.9999 - val_loss: 0.0040 - val_accuracy: 0.9992


In [8]:
model.evaluate(xtest,ytest)
model.save('D_MODEL.keras')

1205/1205 [==============================] - 18s 15ms/step - loss: 0.0028 - accuracy: 0.9995
